In [6]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import os, sys
print(os.getcwd())
sys.path.append('..')
from utils import *
from FAST_LMM.FaST_LMM import FASTLMM
from bslmm_simulation import gemma_operator as gemma
from arguments import get_args
from frequentist_simulation.H_functions import *

e:\OneDrive\Programming\Python\CVc_in_bio_informatics\src\frequentist_simulation
e:\OneDrive\Programming\Python\CVc_in_bio_informatics\src\frequentist_simulation


In [7]:
SNP_file =  '../../data/SNP_in_200GENE_chr1.csv'
from frequentist_simulation.simulation import load
SNPs = load(SNP_file)

In [43]:
from frequentist_simulation.simulation import phenotype_simulator
y  = phenotype_simulator(SNPs)

In [44]:
n = SNPs.shape[0]
data = np.concatenate([np.ones([n, 1]), SNPs], axis=1)
G_tr, G_te, y_tr, y_te = train_test_split(data, y, test_size=0.2)
d = num_fixed_snps = 200
X_tr, X_te = G_tr[:, :d + 1], G_te[:, :d + 1]
W_tr, W_te = G_tr[:, 1:], G_te[:, 1:]
n_tr, sc = W_tr.shape

In [45]:
## using FaST-LMM
fast = FASTLMM(lowRank=True, REML=False)
fast.fit(X_tr, y_tr, 1 / np.sqrt(sc) * W_tr)
sigmas = (fast.sigma_g2, fast.sigma_e2)
print(sigmas)

------------- FAST-LMM------------------
LowRank is set as True, not using REML
Rank of W is 2038, shape of W is (2038, 10000).
--- 26.606565475463867 seconds for SVD calculation ---
Optimization Results:
Delta is calculated as:  6.555589555209418
Maximum Likelihood is calculated as:  -2455.709784891998
---------------Summary------------------
LowRank is set as True, not using REML
Heritability h2: 0.13235234559697875
Sigma_g2: 0.09551526161684576
Sigma_e2: 0.6261588514184891
------ 45.332000494003296 seconds ------
(0.09551526161684576, 0.6261588514184891)


In [46]:
K_relatedness = 1 / sc * W_tr @ W_tr.T
#sigmas = gemma.gemma_var_estimator(y_tr, K_relatedness, 'var_component')
#print(sigmas)
V = sigmas[0] * K_relatedness + sigmas[1] * np.identity(n_tr)
V_inv = inv(V)


In [52]:

n_folds = 10
## calculating H for 10-folds CV
### OLS
H_cv_ols_k = getHcv_for_Kfolds(X_tr, y_tr, H_function_ols, nfolds=n_folds)

First 5 fold indices : [(0, 204), (204, 408), (408, 612), (612, 816), (816, 1020)]


In [47]:
### WLS
H_cv_wls_k = getHcv_for_Kfolds(X_tr,
                                y_tr,
                                H_function_wls,
                                nfolds=n_folds,
                                V=V)

### LMM
H_cv_lmm_k = H_function_lmm(X_tr, H_cv_wls_k, V=V)

#### Ridge
lamb = 100
H_cv_ridge_k = getHcv_for_Kfolds(X_tr,
                                    y_tr,
                                    H_function_ridge,
                                    nfolds=n_folds,
                                    lamb=lamb)

First 5 fold indices : [(0, 204), (204, 408), (408, 612), (612, 816), (816, 1020)]
First 5 fold indices : [(0, 204), (204, 408), (408, 612), (612, 816), (816, 1020)]
First 5 fold indices : [(0, 204), (204, 408), (408, 612), (612, 816), (816, 1020)]
First 5 fold indices : [(0, 204), (204, 408), (408, 612), (612, 816), (816, 1020)]


In [48]:
## get random generated and sampled from X and W
n_tr = X_tr.shape[0]
n_te = X_te.shape[0]
W_te_random = np.random.choice([0, 1, 2], [n_te, sc])
X_te_random = np.concatenate([np.ones([n_te, 1]), W_te_random[:, :d]],
                                axis=1)
samples = np.random.choice(np.arange(n_tr), n_te)
W_te_sample = W_tr[samples, :]
X_te_sample = X_tr[samples, :]

In [50]:
## calculating CV CVc and test error
# Using Test Error to estimate the Generalization error
Error_cv_lmm = 1 / n_tr * (np.sum(np.square(y_tr - H_cv_lmm_k @ y_tr)))
Error_cv_wls = 1 / n_tr * (np.sum(np.square(y_tr - H_cv_wls_k @ y_tr)))
Error_cv_ols = 1 / n_tr * (np.sum(np.square(y_tr - H_cv_ols_k @ y_tr)))
Error_cv_ridge = 1 / n_tr * (np.sum(np.square(y_tr - H_cv_ridge_k @ y_tr)))

In [51]:
Error_cv_wls

3.986125047283216e+62

In [53]:
# using estimated sigma_g2 to get the Covariance(y_tr, y_te)
V_tr_te = 1 / sc * sigmas[0] * W_tr @ W_te.T

H_te_wls = X_te @ inv(X_tr.T @ V_inv @ X_tr) @ X_tr.T @ V_inv
H_te_lmm = H_te_wls + V_tr_te.T @ V_inv @ (
    np.identity(n_tr) - X_tr @ inv(X_tr.T @ V_inv @ X_tr) @ X_tr.T @ V_inv)
H_te_ols = X_te @ inv(X_tr.T @ X_tr) @ X_tr.T
H_te_ridge = X_te @ inv(X_tr.T @ X_tr +
                        lamb * np.identity(X_tr.shape[1])) @ X_tr.T

Error_te_lmm = 1 / n_te * (np.sum(np.square(y_te - H_te_lmm @ y_tr)))
Error_te_wls = 1 / n_te * (np.sum(np.square(y_te - H_te_wls @ y_tr)))
Error_te_ols = 1 / n_te * (np.sum(np.square(y_te - H_te_ols @ y_tr)))
Error_te_ridge = 1 / n_te * (np.sum(np.square(y_te - H_te_ridge @ y_tr)))

Correction_lmm = 2 * (1 / n_tr * np.trace(H_cv_lmm_k @ V) -
                        1 / n_te * np.trace(H_te_lmm @ V_tr_te))
Correction_wls = 2 * (1 / n_tr * np.trace(H_cv_wls_k @ V) -
                        1 / n_te * np.trace(H_te_wls @ V_tr_te))
Correction_ols = 2 * (1 / n_tr * np.trace(H_cv_ols_k @ V) -
                        1 / n_te * np.trace(H_te_ols @ V_tr_te))
Correction_ridge = 2 * (1 / n_tr * np.trace(H_cv_ols_k @ V) -
                        1 / n_te * np.trace(H_te_ols @ V_tr_te))

Error_cv_lmm_c = Error_cv_lmm + Correction_lmm
Error_cv_wls_c = Error_cv_wls + Correction_wls
Error_cv_ols_c = Error_cv_ols + Correction_ols
Error_cv_ridge_c = Error_cv_ridge + Correction_ridge

In [30]:
    # using  generated Test set to correct
    V_tr_te_random = 1 / sc * sigmas[0] * W_tr @ W_te_random.T

    H_te_wls_r = X_te_random @ inv(X_tr.T @ V_inv @ X_tr) @ X_tr.T @ V_inv
    H_te_lmm_r = H_te_wls_r + V_tr_te_random.T @ V_inv @ (
        np.identity(n_tr) - X_tr @ inv(X_tr.T @ V_inv @ X_tr) @ X_tr.T @ V_inv)
    H_te_ols_r = X_te_random @ inv(X_tr.T @ X_tr) @ X_tr.T
    H_te_ridge_r = X_te_random @ inv(X_tr.T @ X_tr + lamb *
                                     np.identity(X_tr.shape[1])) @ X_tr.T

    Correction_lmm_r = 2 * (1 / n_tr * np.trace(H_cv_lmm_k @ V) -
                            1 / n_te * np.trace(H_te_lmm_r @ V_tr_te_random))
    Correction_wls_r = 2 * (1 / n_tr * np.trace(H_cv_wls_k @ V) -
                            1 / n_te * np.trace(H_te_wls_r @ V_tr_te_random))
    Correction_ols_r = 2 * (1 / n_tr * np.trace(H_cv_ols_k @ V) -
                            1 / n_te * np.trace(H_te_ols_r @ V_tr_te_random))
    Correction_ridge_r = 2 * (1 / n_tr * np.trace(H_cv_ols_k @ V) - 1 / n_te *
                              np.trace(H_te_ridge_r @ V_tr_te_random))

    Error_cv_lmm_c_r = Error_cv_lmm + Correction_lmm_r
    Error_cv_wls_c_r = Error_cv_wls + Correction_wls_r
    Error_cv_ols_c_r = Error_cv_ols + Correction_ols_r
    Error_cv_ridge_c_r = Error_cv_ridge + Correction_ridge_r

In [31]:
Error_cv_lmm_c_r

0.9585450392922745

In [32]:
    # using sampled dataset from training data to correct
    V_tr_te_sample = 1 / sc * sigmas[0] * W_tr @ W_te_sample.T

    H_te_wls_s = X_te_sample @ inv(X_tr.T @ V_inv @ X_tr) @ X_tr.T @ V_inv
    H_te_lmm_s = H_te_wls_s + V_tr_te_sample.T @ V_inv @ (
        np.identity(n_tr) - X_tr @ inv(X_tr.T @ V_inv @ X_tr) @ X_tr.T @ V_inv)
    H_te_ols_s = X_te_sample @ inv(X_tr.T @ X_tr) @ X_tr.T
    H_te_ridge_s = X_te_sample @ inv(X_tr.T @ X_tr + lamb *
                                     np.identity(X_tr.shape[1])) @ X_tr.T

    Correction_lmm_s = 2 * (1 / n_tr * np.trace(H_cv_lmm_k @ V) -
                            1 / n_te * np.trace(H_te_lmm_s @ V_tr_te_sample))
    Correction_wls_s = 2 * (1 / n_tr * np.trace(H_cv_wls_k @ V) -
                            1 / n_te * np.trace(H_te_wls_s @ V_tr_te_sample))
    Correction_ols_s = 2 * (1 / n_tr * np.trace(H_cv_ols_k @ V) -
                            1 / n_te * np.trace(H_te_ols_s @ V_tr_te_sample))
    Correction_ridge_s = 2 * (1 / n_tr * np.trace(H_cv_ols_k @ V) - 1 / n_te *
                              np.trace(H_te_ridge_s @ V_tr_te_sample))

    Error_cv_lmm_c_s = Error_cv_lmm + Correction_lmm_s
    Error_cv_wls_c_s = Error_cv_wls + Correction_wls_s
    Error_cv_ols_c_s = Error_cv_ols + Correction_ols_s
    Error_cv_ridge_c_s = Error_cv_ridge + Correction_ridge_s

In [33]:
Error_cv_lmm_c_s

0.6226750179773686